# Lesson 1 - Simple LLM call using langchain
This is the basic lesson to show you how to invoke an LLM using langchain

In [1]:
# We first install the packages
%pip install langchain langchain-openai langchain-community --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
# We install our envirnment variables helper library
%pip install python-dotenv --quiet

# We load the library and the environment variables
import dotenv
dotenv.load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

In [3]:
# We load the settings
import _settings

# The challenge prompt is the challenge we want our agents to solve
# You can change it in the _settings.py file
# Be sure to restart ^^ your kernel after changing the settings
challenge_prompt = _settings.CHALLENGE_PROMPT
print(challenge_prompt)

Write me a shell script to list the files in the current directory. 
Output the script as markdown code block


In [4]:
# We load the llm language model from _models
from langchain_openai import OpenAI
llm = OpenAI()

# We invoke the language model with the challenge prompt
result = llm.invoke(_settings.CHALLENGE_PROMPT)

# We print the result
print(result)




```
#!/bin/bash

# This script lists all the files in the current directory

# Change directory to current working directory
cd "$(pwd)"

# Print heading
echo "Files in current directory:"

# Loop through all files in current directory
for file in *; do
    # Check if file is a regular file
    if [ -f "$file" ]; then
        # Print file name
        echo "$file"
    fi
done
```

To use this script, save it in a file with a .sh extension and make it executable using the command `chmod +x <filename.sh>`. Then, run the script using `./<filename.sh>`. The output will be a list of all the files in the current directory.


To see what is going of the wire to the LLM and back, we add a callback handler that prints this out.

In [5]:
# We import the callback handler from _callbacks
from _callbacks import PrettyPrintCallbackHandler
callback = PrettyPrintCallbackHandler()

# Now we can do the same call
# but this time we will use the callback handler
# That will print out the intermediate results
# We set the streaming parameter to True to see how the callback works

# To make sure we always the same result we set the temperature to 0
# Think of reducing the creative randomness of the llm
llm = OpenAI(streaming=True, temperature=0 , callbacks=[callback])
result=llm.invoke(challenge_prompt)



[llm][start] - prompts: Write me a shell script to list the files in the current directory. 
Output the script as markdown code block

[llm][end] - generation 

```
#!/bin/bash

# This script lists all the files in the current directory

# Change directory to current directory
cd .

# Loop through all files in current directory
for file in *
do
    # Check if file is a regular file
    if [ -f "$file" ]
    then
        # Print file name
        echo "$file"
    fi
done
```

To run this script, save it as a .sh file and execute it in the terminal using the command `./script.sh`.


In [6]:
# We print the final result
# And get the same result as before
print(result)



```
#!/bin/bash

# This script lists all the files in the current directory

# Change directory to current directory
cd .

# Loop through all files in current directory
for file in *
do
    # Check if file is a regular file
    if [ -f "$file" ]
    then
        # Print file name
        echo "$file"
    fi
done
```

To run this script, save it as a .sh file and execute it in the terminal using the command `./script.sh`.


In [7]:
# Now to avoid we need to this every time in a notebook
# We can put this in a function in _models

# The default model is set in _settings.py
import _models, _settings
# We print the model name from settings
print(_settings.MODEL_NAME)

gpt-4


In [8]:

# We pass this model name to the get_llm helper function
llm = _models.get_llm(_settings.MODEL_NAME,callbacks=[callback])
result=llm.invoke(challenge_prompt)

# We print the final result
print(result)

[chat_model][start] - prompts : Write me a shell script to list the files in the current directory. 
Output the script as markdown code block

[llm][end] - generation Here is a simple shell script to list the files in the current directory:

```bash
#!/bin/bash
# This script will list the files in the current directory

for file in ./*
do
  if [ -f "$file" ]; then
    echo "$file"
  fi
done
```

This script uses a for loop to iterate over all items in the current directory. The `if [ -f "$file" ]` line checks if the item is a file (not a directory), and if it is, it prints the name of the file.
content='Here is a simple shell script to list the files in the current directory:\n\n```bash\n#!/bin/bash\n# This script will list the files in the current directory\n\nfor file in ./*\ndo\n  if [ -f "$file" ]; then\n    echo "$file"\n  fi\ndone\n```\n\nThis script uses a for loop to iterate over all items in the current directory. The `if [ -f "$file" ]` line checks if the item is a file (not 

In [9]:
# For reference you can see the whole helper _models.py file
%pycat _models.py 

# WARNING: When changing something here , make sure you restart the notebook kernel
from langchain_openai import OpenAI, ChatOpenAI
from langchain_community.chat_models.ollama import ChatOllama

def get_llm(name, callbacks=[]):
    llm = OpenAI(streaming=True, temperature=0 , callbacks=callbacks)

    openai_llm = ChatOpenAI(
        model="gpt-4",
        temperature=0,
        callbacks=callbacks,
        streaming=False
    )

    #if name == "ollama":
    ollama_llm = ChatOllama(
        model="llama3",
        temperature=0,
        callbacks=callbacks,
        streaming=False
    )

    llm = ollama_llm
    llm = openai_llm
    return llm
